In [1]:
!date

Sun Mar  6 19:13:14 PST 2022


This notebook uses [Stan](https://mc-stan.org) and [Bean Machine](https://beanmachine.org) to fit a linear model with outliers. The model and data are taken from Jake VanderPlas's post [Frequentism and Bayesianism II: When Results Differ](http://jakevdp.github.io/blog/2014/06/06/frequentism-and-bayesianism-2-when-results-differ/). Specifically, this is [Example #2](http://jakevdp.github.io/blog/2014/06/06/frequentism-and-bayesianism-2-when-results-differ/#Example-#2:-Linear-Fit-with-Outliers) from his post. There's also a very nice discussion of Bayesian modeling for handling outliers in §8.3 of [Data Analysis - A Bayesian Tutorial](https://www.goodreads.com/book/show/1430545.Data_Analysis) by D.S. Sivia. Finally, this discussion is complementary to the official Bean Machine [Robust Linear Regression](https://beanmachine.org/docs/tutorials/#robust-linear-regression) tutorial.

Points are outliers with probability $\theta$. All observations are drawn from a normal distribution. For points that are not outliers, we use the estimated observation error $e_i$ as the standard deviation. For outliers, we use a constant standard deviation $\sigma_o$, which is estimated as part of the model. The model description below, $z$ is a latent indicator for whether or not the point is an outlier.

$$
\begin{align}
\theta & \sim \mathrm{Beta}(2, 5) \\
\sigma_o & \sim \mathrm{Gamma}(2, 2) \\
\beta_0 & \sim \mathcal{N}(0, 10) \\
\beta_1 & \sim \mathcal{N}(0, 10) \\
z & \sim \mathrm{Bernouilli}(\theta) \\
y & \sim \mathcal{N}(\beta_0 + \beta_1 x, \sigma_{z[i]}) \\
\end{align}
$$

where

$$
\sigma_{z[i]} =
    \begin{cases}
    e_i, & z_i = 0 \\
    \sigma_o, & z_i = 1 \\
    \end{cases}
$$

For both the Bean Machine and Stan implementations, we marginalize out the latent discrete variable $z$, as described [here](https://mc-stan.org/docs/2_29/stan-users-guide/summing-out-the-responsibility-parameter.html) in the Stan documentation.

$$
p(y_i | \theta, \beta_0, \beta_1, \sigma_o) = \theta \mathcal{N}(\beta_0 + \beta_1 x_i, \sigma_o) + (1 - \theta) \mathcal{N}(\beta_0 + \beta_1 x_i, e_i)
$$

In [2]:
# This is needed for PyStan to run in a Jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [3]:
import altair
import beanmachine.ppl as bm
import numpy as np
import pandas as pd
import stan
import torch
import torch.distributions as dist
from beanmachine.ppl.inference.bmg_inference import BMGInference

In [4]:
altair.renderers.enable("jupyterlab")

RendererRegistry.enable('jupyterlab')

In [5]:
BLUE = "#4e79a7"
ORANGE = "#f28e2b"
RED = "#e15759"
CYAN = "#76b7b2"
GREEN = "#59a14f"
YELLOW = "#edc948"
PURPLE = "#b07aa1"
PINK = "#ff9da7"
BROWN = "#9c755f"
GRAY = "#bab0ac"

In [6]:
def calc_prob_outlier(
    x: torch.Tensor,
    y: torch.Tensor,
    err: torch.Tensor,
    beta_0: float,
    beta_1: float,
    theta: float,
    sigma_o: float,
) -> torch.Tensor:
    """Calculate the probabilities that the points are outliers.
    
    See https://mc-stan.org/docs/2_29/stan-users-guide/summing-out-the-responsibility-parameter.html#recovering-posterior-mixture-proportions

    Note that I'm computing some sort of best estimate for the probabilities. That is,
    I'm not propagating uncertainty (as represented by samples of the parameters).
    """
    mu = beta_0 + beta_1 * x
    prob_z1 = dist.Normal(mu, err).log_prob(y).exp() * (1 - theta)
    prob_z2 = dist.Normal(mu, sigma_o).log_prob(y).exp() * theta
    return prob_z2 / (prob_z1 + prob_z2)


In [7]:
x = torch.tensor(
    [0, 3, 9, 14, 15, 19, 20, 21, 30, 35, 40, 41, 42, 43, 54, 56, 67, 69, 72, 88],
    dtype=torch.float32,
)
y = torch.tensor(
    [33, 68, 34, 34, 37, 71, 37, 44, 48, 49, 53, 49, 50, 48, 56, 60, 61, 63, 44, 71],
    dtype=torch.float32,
)
err = torch.tensor(
    [
        3.6,
        3.9,
        2.6,
        3.4,
        3.8,
        3.8,
        2.2,
        2.1,
        2.3,
        3.8,
        2.2,
        2.8,
        3.9,
        3.1,
        3.4,
        2.6,
        3.4,
        3.7,
        2.0,
        3.5,
    ]
)


In [8]:
df_obs = pd.DataFrame({"x": x, "y": y, "err": err})

In [9]:
error_bars = (
    altair.Chart(df_obs)
    .mark_errorbar(color=BLUE)
    .encode(altair.X("x"), altair.Y("y"), altair.YError("err"))
)
points = (
    altair.Chart(df_obs)
    .mark_circle(color=BLUE)
    .encode(x=altair.X("x"), y=altair.Y("y", scale=altair.Scale(zero=False)))
)
points + error_bars


<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [10]:
STAN_CODE_NAIVE = """
data {
    int<lower=0> n_obs;
    vector[n_obs] x;
    vector[n_obs] y;
    vector[n_obs] err;
}

parameters {
    real beta_0;
    real beta_1;
}

model {
    beta_0 ~ normal(0, 10);
    beta_1 ~ normal(0, 10);

    y ~ normal(beta_0 + beta_1 * x, err);
}
"""

In [11]:
posterior = stan.build(
    program_code=STAN_CODE_NAIVE,
    data={"n_obs": len(x), "x": x.tolist(), "y": y.tolist(), "err": err.tolist()},
    random_seed=332211,
)

Building...



Building: found in cache, done.

In [12]:
fit = posterior.sample(num_chains=4, num_samples=1000)

Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 2.5e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.25 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 7e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.07 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.05 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.05 seconds.
  Adjust your expectations accordingly!


In [13]:
df_samples = fit.to_frame()

In [14]:
df_samples.describe(percentiles=[0.05, 0.5, 0.95]).T

,count,mean,std,min,5%,50%,95%,max
parameters,,,,,,,,
lp__,4000.0,-109.513387,1.050401,-116.919819,-111.616797,-109.188672,-108.519086,-108.472036
accept_stat__,4000.0,0.923421,0.107637,0.251288,0.696539,0.969704,1.000000,1.000000
stepsize__,4000.0,0.422027,0.031133,0.382050,0.382050,0.421039,0.463980,0.463980
treedepth__,4000.0,2.210000,0.720090,1.000000,1.000000,2.000000,3.000000,4.000000
n_leapfrog__,4000.0,6.114500,3.744649,1.000000,1.000000,7.000000,15.000000,15.000000
divergent__,4000.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
energy__,4000.0,110.523268,1.444446,108.487704,108.829640,110.203549,113.405848,118.316766
beta_0,4000.0,39.042598,1.249936,34.444524,36.958583,39.059034,41.032308,42.658000
beta_1,4000.0,0.249544,0.028240,0.169302,0.204554,0.248115,0.297175,0.360917


In [15]:
error_bars = (
    altair.Chart(df_obs)
    .mark_errorbar(color=BLUE)
    .encode(altair.X("x"), altair.Y("y"), altair.YError("err"))
)
points = (
    altair.Chart(df_obs)
    .mark_circle(color=BLUE)
    .encode(x=altair.X("x"), y=altair.Y("y", scale=altair.Scale(zero=False)))
)
best_fit = (
    altair.Chart(
        pd.DataFrame({"x": np.linspace(0, 100, 101)}).assign(
            y=lambda df: df_samples["beta_0"].mean()
            + df_samples["beta_1"].mean() * df["x"]
        )
    )
    .mark_line(color=RED)
    .encode(altair.X("x"), altair.Y("y"))
)
points + error_bars + best_fit


<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [16]:
STAN_CODE_OUTLIERS = """
data {
    int<lower=0> n_obs;
    vector[n_obs] x;
    vector[n_obs] y;
    vector[n_obs] err;
}

parameters {
    real beta_0;
    real beta_1;
    real<lower=0> sigma_o;
    real<lower=0, upper=1> theta;
}

model {
    real mu;
    beta_0 ~ normal(0, 10);
    beta_1 ~ normal(0, 10);
    sigma_o ~ gamma(2, 2);
    theta ~ beta(2, 5);

    for (i in 1:n_obs) {
        mu = beta_0 + beta_1 * x[i];
        target += log_mix(theta, normal_lpdf(y[i] | mu, sigma_o ), normal_lpdf(y[i] | mu, err[i] ));
    }
}
"""

In [17]:
posterior = stan.build(
    program_code=STAN_CODE_OUTLIERS,
    data={"n_obs": len(x), "x": x.tolist(), "y": y.tolist(), "err": err.tolist()},
    random_seed=332211,
)


Building...



Building: found in cache, done.Messages from stanc:
Warning in '/var/folders/9c/th30_62d0lx78835_vdd37sr0000gn/T/httpstan_e528pokx/model_htp43kdw.stan', line 21, column 4: The
    parameter theta is on the left-hand side of more than one tilde
    statement.


In [18]:
fit = posterior.sample(num_chains=4, num_samples=1000)

Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 2e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.2 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 2.9e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.29 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.2e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.12 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 6e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.06 seconds.
  Adjust your expectations accordingly!


In [19]:
df_samples = fit.to_frame()

In [20]:
df_summary = df_samples.describe(percentiles=[0.05, 0.5, 0.95])
df_summary.T

,count,mean,std,min,5%,50%,95%,max
parameters,,,,,,,,
lp__,4000.0,-96.847199,1.506942,-107.462114,-99.743475,-96.512814,-95.069789,-94.743005
accept_stat__,4000.0,0.929085,0.098290,0.136147,0.730945,0.966989,1.000000,1.000000
stepsize__,4000.0,0.337962,0.011483,0.320605,0.320605,0.340878,0.349488,0.349488
treedepth__,4000.0,3.104250,0.693904,1.000000,2.000000,3.000000,4.000000,4.000000
n_leapfrog__,4000.0,10.642000,4.507873,1.000000,3.000000,11.000000,15.000000,31.000000
divergent__,4000.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
energy__,4000.0,98.864675,2.049025,94.910001,96.098780,98.561324,102.544776,112.184034
beta_0,4000.0,32.198265,1.863560,25.534571,29.322431,32.081496,35.431268,42.271615
beta_1,4000.0,0.447492,0.041893,0.267811,0.376613,0.448666,0.514474,0.606398


In [21]:
df_params_mean = df_summary.loc["mean"]

In [22]:
prob_outlier = calc_prob_outlier(
    x,
    y,
    err,
    beta_0=df_params_mean["beta_0"],
    beta_1=df_params_mean["beta_1"],
    theta=df_params_mean["theta"],
    sigma_o=df_params_mean["sigma_o"],
)


In [23]:
error_bars = (
    altair.Chart()
    .mark_errorbar()
    .encode(
        altair.X("x"), altair.Y("y"), altair.YError("err"), altair.Color("is_outlier")
    )
)
points = (
    altair.Chart()
    .mark_circle()
    .encode(
        altair.X("x"),
        altair.Y("y", scale=altair.Scale(zero=False)),
        altair.Color("is_outlier"),
    )
)
altair.layer(
    points,
    error_bars,
    data=pd.DataFrame({"x": x, "y": y, "err": err, "is_outlier": prob_outlier > 0.5}),
) + (
    altair.Chart(
        pd.DataFrame({"x": np.linspace(0, 100, 101)}).assign(
            y=lambda df: df_params_mean["beta_0"] + df_params_mean["beta_1"] * df["x"]
        )
    )
    .mark_line(color=RED)
    .encode(altair.X("x"), altair.Y("y"))
)

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


# Bean Machine

In [24]:
@bm.random_variable
def beta_0() -> bm.RVIdentifier:
    return dist.Normal(0, 10)


@bm.random_variable
def beta_1() -> bm.RVIdentifier:
    return dist.Normal(0, 10)


@bm.random_variable
def sigma_o() -> bm.RVIdentifier:
    return dist.Gamma(1, 1)


@bm.random_variable
def theta() -> bm.RVIdentifier:
    return dist.Beta(2, 5)


@bm.functional
def likelihood(i: int) -> float:
    mu = beta_0() + beta_1() * x[i]
    ll1 = torch.log(theta()) + dist.Normal(mu, sigma_o()).log_prob(y[i])
    ll2 = torch.log(1 - theta()) + dist.Normal(mu, err[i]).log_prob(y[i])
    return torch.logaddexp(ll1, ll2).exp()


@bm.random_variable
def d(i: int) -> bm.RVIdentifier:
    return dist.Bernoulli(likelihood(i))


In [25]:
altair.Chart(
    pd.DataFrame().assign(
        theta=torch.arange(0, 1, 0.01),
        p=lambda df: dist.Beta(2, 5).log_prob(torch.tensor(df["theta"])).exp(),
    )
).mark_line().encode(x="theta", y="p")


<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [26]:
queries = [beta_0(), beta_1(), sigma_o(), theta()]
observations = {d(i): torch.tensor(1.0) for i in range(len(x))}

In [27]:
# samples = BMGInference().infer(
#     queries=queries,
#     observations=observations,
#     num_samples=4000,
#     num_chains=4,
# )

In [28]:
samples = bm.GlobalNoUTurnSampler().infer(
    queries=queries,
    observations=observations,
    num_samples=4000,
    num_adaptive_samples=4000,
    num_chains=1,
)

Samples collected: 100%|██████████| 8000/8000 [14:12<00:00,  9.38it/s] 


In [29]:
df_summary = (
    bm.Diagnostics(samples)
    .summary()
    .set_index(pd.Series(["beta_0", "beta_1", "sigma_o", "theta"]))
)


In [30]:
df_params_mean = df_summary["avg"]

In [31]:
prob_outlier = calc_prob_outlier(
    x,
    y,
    err,
    beta_0=df_params_mean["beta_0"],
    beta_1=df_params_mean["beta_1"],
    theta=df_params_mean["theta"],
    sigma_o=df_params_mean["sigma_o"],
)

In [32]:
error_bars = (
    altair.Chart()
    .mark_errorbar()
    .encode(
        altair.X("x"), altair.Y("y"), altair.YError("err"), altair.Color("is_outlier")
    )
)
points = (
    altair.Chart()
    .mark_circle()
    .encode(
        altair.X("x"),
        altair.Y("y", scale=altair.Scale(zero=False)),
        altair.Color("is_outlier"),
    )
)
altair.layer(
    points,
    error_bars,
    data=pd.DataFrame({"x": x, "y": y, "err": err, "is_outlier": prob_outlier > 0.5}),
) + (
    altair.Chart(
        pd.DataFrame({"x": np.linspace(0, 100, 101)}).assign(
            y=lambda df: samples.get_variable(beta_0()).squeeze().mean().item()
            + samples.get_variable(beta_1()).squeeze().mean().item() * df["x"]
        )
    )
    .mark_line(color=RED)
    .encode(altair.X("x"), altair.Y("y"))
)


<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html
